In [43]:
############# TRATATIVA E NORMALIZAÇÃO DOS DADOS ######################
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV 
import joblib 
import os 

df_obesity = pd.read_csv("C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\data\\Obesity.csv")


# Passos de exploração baixo estão descrito no Exploracao.ipynb
novos_nomes = {
    'Gender': 'Gênero',
    'Age': 'Idade',
    'Height': 'Altura',
    'Weight': 'Peso',
    'family_history': 'Histórico_Familiar_Obesidade',
    'FAVC': 'Frequencia_Consumo_Alimento_Calorico',
    'FCVC': 'Frequencia_Consumo_Vegetais',
    'NCP': 'Numero_Refeicoes_Principais',
    'CAEC': 'Consumo_Alimento_Entre_Refeicoes',
    'SMOKE': 'Fumante',
    'CH2O': 'Consumo_Agua',
    'SCC': 'Monitoramento_Calorico',
    'FAF': 'Frequencia_Atividade_Fisica',
    'TUE': 'Tempo_Uso_Tecnologia',
    'CALC': 'Consumo_Alcool',
    'MTRANS': 'Meio_Transporte',
    'Obesity': 'Status_Obesidade'  
}

df_obesity = df_obesity.rename(columns=novos_nomes)

#Colunas de sim e nao para 1 e 0
colunas_sim_nao = ['Histórico_Familiar_Obesidade', 'Frequencia_Consumo_Alimento_Calorico', 'Fumante', 'Monitoramento_Calorico']

dict_sin_nao = {'yes': 1, 'no': 0}

df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)




#coluna gênero para 0 e 1

coluna_genero = ['Gênero']
dic_genero = {'Female' : 0,'Male': 1}

df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)



#colunas com classificacao de 0 a 3 para valores numericos
colunas_class_0_3 = ['Consumo_Alcool','Consumo_Alimento_Entre_Refeicoes']
dic_class_0_3 = {'no': 0,'Sometimes': 1,'Frequently': 2, 'Always': 3}

df_obesity[colunas_class_0_3] = df_obesity[colunas_class_0_3].replace(dic_class_0_3).astype(int)


#colunas com classificacao de trasportes dividida em 3 
# 0 para baixa intensidade 
# 1 para meia intensidade
# 2 para alta intensidade

colunas_trasporte = ['Meio_Transporte']
dic_trasporte = {'Walking': 2,'Public_Transportation': 1,'Automobile': 0,
                 'Motorbike': 0,'Bike': 2}

df_obesity[colunas_trasporte] = df_obesity[colunas_trasporte].replace(dic_trasporte).astype(int)



### Colunas onde ponto flutuante está incorreto, pegando o primeiro digito

colunas_primeiro_digito = [ 'Tempo_Uso_Tecnologia','Frequencia_Atividade_Fisica',
'Consumo_Agua','Numero_Refeicoes_Principais','Frequencia_Consumo_Vegetais']

for coluna in colunas_primeiro_digito:
    df_obesity[coluna] = df_obesity[coluna].astype(str).str[0]

    df_obesity[coluna] = df_obesity[coluna].astype(int)

# Coluna de idade arredondado
df_obesity['Idade'] = df_obesity['Idade'].astype(int)

#Colunas de peso e altura 2 casa decimais
df_obesity['Peso'] = df_obesity['Peso'].astype(float).round(2)
df_obesity['Altura'] = df_obesity['Altura'].astype(float).round(2)


#IREMOS TESTAR 3 ABORDAGENS 

C:\Users\Igor\AppData\Local\Temp\ipykernel_15340\3318934538.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)
C:\Users\Igor\AppData\Local\Temp\ipykernel_15340\3318934538.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)
C:\Users\Igor\AppData\Local\Temp\ipykernel_15340\3318934538.py:65: FutureWarning: Downcasting behavior in `replace` is deprecated and will be 

In [44]:

# 1- CLASSIFICANDO OBESIDADE EM 2 NIVEIS: OBESO E NAO OBESO

#Coluna de classificacao de obesiade para valor numerico

df_obesity_biaria = df_obesity

def normalize_obesity_binario(level):
    if level in ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II']:
        return 0  # Não obeso
    else:
        return 1  # Obeso

df_obesity_biaria["Status_Obesidade"] = df_obesity_biaria["Status_Obesidade"].apply(normalize_obesity_binario)

df_obesity_biaria.tail(10)

,Gênero,Idade,Altura,Peso,Histórico_Familiar_Obesidade,Frequencia_Consumo_Alimento_Calorico,Frequencia_Consumo_Vegetais,Numero_Refeicoes_Principais,Consumo_Alimento_Entre_Refeicoes,Fumante,Consumo_Agua,Monitoramento_Calorico,Frequencia_Atividade_Fisica,Tempo_Uso_Tecnologia,Consumo_Alcool,Meio_Transporte,Status_Obesidade
2101,0,25,1.63,107.22,1,1,3,3,1,0,2,0,0,0,1,1,1
2102,0,25,1.63,108.11,1,1,3,3,1,0,2,0,0,0,1,1,1
2103,0,21,1.72,133.03,1,1,3,3,1,0,1,0,1,0,1,1,1
2104,0,21,1.73,133.04,1,1,3,3,1,0,1,0,1,0,1,1,1
2105,0,21,1.73,131.34,1,1,3,3,1,0,1,0,1,0,1,1,1
2106,0,20,1.71,131.41,1,1,3,3,1,0,1,0,1,0,1,1,1
2107,0,21,1.75,133.74,1,1,3,3,1,0,2,0,1,0,1,1,1
2108,0,22,1.75,133.69,1,1,3,3,1,0,2,0,1,0,1,1,1
2109,0,24,1.74,133.35,1,1,3,3,1,0,2,0,1,0,1,1,1
2110,0,23,1.74,133.47,1,1,3,3,1,0,2,0,1,0,1,1,1


In [45]:

# 2- CLASSIFICANDO OBESIDADE EM 3 NIVEIS: NORMAL, SOBREPESO E OBESO

#Coluna de classificacao de obesiade para valor numerico

df_obesity_3_class = df_obesity

def normalize_obesity_3(level):
    if level in ['Insufficient_Weight', 'Normal_Weight']:
        return 0  # Peso normal
    elif level in ['Overweight_Level_I', 'Overweight_Level_II']:
        return 1  # Sobrepeso
    else:
        return 2  # Obeso

df_obesity_3_class["Status_Obesidade"] = df_obesity_3_class["Status_Obesidade"].apply(normalize_obesity_3)


df_obesity_biaria.tail(10)

,Gênero,Idade,Altura,Peso,Histórico_Familiar_Obesidade,Frequencia_Consumo_Alimento_Calorico,Frequencia_Consumo_Vegetais,Numero_Refeicoes_Principais,Consumo_Alimento_Entre_Refeicoes,Fumante,Consumo_Agua,Monitoramento_Calorico,Frequencia_Atividade_Fisica,Tempo_Uso_Tecnologia,Consumo_Alcool,Meio_Transporte,Status_Obesidade
2101,0,25,1.63,107.22,1,1,3,3,1,0,2,0,0,0,1,1,2
2102,0,25,1.63,108.11,1,1,3,3,1,0,2,0,0,0,1,1,2
2103,0,21,1.72,133.03,1,1,3,3,1,0,1,0,1,0,1,1,2
2104,0,21,1.73,133.04,1,1,3,3,1,0,1,0,1,0,1,1,2
2105,0,21,1.73,131.34,1,1,3,3,1,0,1,0,1,0,1,1,2
2106,0,20,1.71,131.41,1,1,3,3,1,0,1,0,1,0,1,1,2
2107,0,21,1.75,133.74,1,1,3,3,1,0,2,0,1,0,1,1,2
2108,0,22,1.75,133.69,1,1,3,3,1,0,2,0,1,0,1,1,2
2109,0,24,1.74,133.35,1,1,3,3,1,0,2,0,1,0,1,1,2
2110,0,23,1.74,133.47,1,1,3,3,1,0,2,0,1,0,1,1,2


In [46]:

# 3- CLASSIFICANDO OBESIDADE EM 4 NIVEIS: ABAIXO DO PESO, NORMAL, SOBREPESO E OBESO

#Coluna de classificacao de obesiade para valor numerico

df_obesity_4_class = df_obesity

def normalize_obesity_4(level):
    if level in ['Insufficient_Weight']:
        return 0  # Abaxio do peso
    elif level in ['Normal_Weight']:
        return 1  # Peso normal
    elif level in ['Overweight_Level_I', 'Overweight_Level_II']:
        return 2  # Sobrepeso
    else:
        return 3  # Obeso

df_obesity_4_class["Status_Obesidade"] = df_obesity_4_class["Status_Obesidade"].apply(normalize_obesity_4)

df_obesity_biaria.tail(10)

,Gênero,Idade,Altura,Peso,Histórico_Familiar_Obesidade,Frequencia_Consumo_Alimento_Calorico,Frequencia_Consumo_Vegetais,Numero_Refeicoes_Principais,Consumo_Alimento_Entre_Refeicoes,Fumante,Consumo_Agua,Monitoramento_Calorico,Frequencia_Atividade_Fisica,Tempo_Uso_Tecnologia,Consumo_Alcool,Meio_Transporte,Status_Obesidade
2101,0,25,1.63,107.22,1,1,3,3,1,0,2,0,0,0,1,1,3
2102,0,25,1.63,108.11,1,1,3,3,1,0,2,0,0,0,1,1,3
2103,0,21,1.72,133.03,1,1,3,3,1,0,1,0,1,0,1,1,3
2104,0,21,1.73,133.04,1,1,3,3,1,0,1,0,1,0,1,1,3
2105,0,21,1.73,131.34,1,1,3,3,1,0,1,0,1,0,1,1,3
2106,0,20,1.71,131.41,1,1,3,3,1,0,1,0,1,0,1,1,3
2107,0,21,1.75,133.74,1,1,3,3,1,0,2,0,1,0,1,1,3
2108,0,22,1.75,133.69,1,1,3,3,1,0,2,0,1,0,1,1,3
2109,0,24,1.74,133.35,1,1,3,3,1,0,2,0,1,0,1,1,3
2110,0,23,1.74,133.47,1,1,3,3,1,0,2,0,1,0,1,1,3
